## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'WeatherPy_Database'

In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Cap Malheureux,-19.9842,57.6142,80.01,76,6,5.99,MU,2021-01-14 02:46:37
11,11,Rikitea,-23.1203,-134.9692,77.74,76,89,12.28,PF,2021-01-14 02:45:58
16,16,Kavieng,-2.5744,150.7967,85.44,68,68,13.44,PG,2021-01-14 02:46:37
20,20,Miri,4.4148,114.0089,78.80,94,40,4.61,MY,2021-01-14 02:46:38
22,22,Avarua,-21.2078,-159.7750,84.20,70,75,19.57,CK,2021-01-14 02:44:24
24,24,Georgetown,5.4112,100.3354,81.00,78,20,1.14,MY,2021-01-14 02:42:03
26,26,Ugoofaaru,5.6667,73.0000,80.10,75,97,2.55,MV,2021-01-14 02:46:38
28,28,Arraial Do Cabo,-22.9661,-42.0278,75.61,89,20,18.63,BR,2021-01-14 02:46:38
30,30,Atuona,-9.8000,-139.0333,77.85,77,13,13.51,PF,2021-01-14 02:46:38
33,33,Alofi,-19.0595,-169.9187,86.00,70,75,11.50,NU,2021-01-14 02:46:38


In [33]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

164


City_ID       164
City          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
Country       163
Date          164
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Cap Malheureux,MU,80.01,-19.9842,57.6142,
11,Rikitea,PF,77.74,-23.1203,-134.9692,
16,Kavieng,PG,85.44,-2.5744,150.7967,
20,Miri,MY,78.80,4.4148,114.0089,
22,Avarua,CK,84.20,-21.2078,-159.7750,
24,Georgetown,MY,81.00,5.4112,100.3354,
26,Ugoofaaru,MV,80.10,5.6667,73.0000,
28,Arraial Do Cabo,BR,75.61,-22.9661,-42.0278,
30,Atuona,PF,77.85,-9.8000,-139.0333,
33,Alofi,NU,86.00,-19.0595,-169.9187,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(15)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Cap Malheureux,MU,80.01,-19.9842,57.6142,Pereybere Hotel & Spa
11,Rikitea,PF,77.74,-23.1203,-134.9692,Pension Maro'i
16,Kavieng,PG,85.44,-2.5744,150.7967,Nusa Island Retreat
20,Miri,MY,78.80,4.4148,114.0089,"Mega Hotel, Miri"
22,Avarua,CK,84.20,-21.2078,-159.7750,Paradise Inn
24,Georgetown,MY,81.00,5.4112,100.3354,Cititel Penang
26,Ugoofaaru,MV,80.10,5.6667,73.0000,Ungoofaaru Inn
28,Arraial Do Cabo,BR,75.61,-22.9661,-42.0278,Pousada Porto Praia
30,Atuona,PF,77.85,-9.8000,-139.0333,Villa Enata
33,Alofi,NU,86.00,-19.0595,-169.9187,Taloa Heights


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))